# Learning: Building a Custom Controller

This notebook guides you through building your own control algorithm. **You'll need to fill in the missing code!**

## What You'll Learn

1. Understanding the controller interface (`BaseController`)
2. Implementing a simple proportional controller
3. Implementing a lookahead-based controller
4. Testing your implementation

## Instructions

- Read each section carefully
- Look for `# TODO:` comments - these indicate where you need to write code
- Fill in the `...` placeholders
- Run cells as you complete them to test your work

## Prerequisites

- Understanding of controllers (see [Building Controller](../building/building_controller.ipynb))
- Understanding of car dynamics
- Basic control theory knowledge (helpful but not required)

In [ ]:
import sys

sys.path.insert(0, '../../src')

import matplotlib.pyplot as plt
import numpy as np

from simple_autonomous_car import (
    BaseController,
    Car,
    CarState,
    Track,
    TrackPlanner,
)

## Step 1: Understanding the BaseController Interface

All controllers must inherit from `BaseController` and implement `compute_control()`.

In [ ]:
from simple_autonomous_car.control.base_controller import BaseController

print("BaseController interface:")
print("  compute_control(car_state, perception_data, costmap, plan, dt) -> Dict")
print("  Returns: {'acceleration': float, 'steering_rate': float}")
print("\nKey points:")
print("  - Must return dict with 'acceleration' and 'steering_rate'")
print("  - Can use plan, perception_data, or costmap (all optional)")
print("  - Should check if enabled before computing control")

## Step 2: Build a Simple Proportional Controller

Let's start with a simple controller that steers towards the next waypoint.

In [ ]:
class SimpleProportionalController(BaseController):
    """
    A simple proportional controller that steers towards the next waypoint.

    TODO: Fill in the implementation!
    """

    def __init__(
        self,
        steering_gain: float = 2.0,
        velocity_gain: float = 0.5,
        target_velocity: float = 10.0,
        name: str = "simple_proportional",
    ):
        """
        Initialize the controller.

        TODO:
        1. Call super().__init__(name=name)
        2. Store steering_gain, velocity_gain, target_velocity
        """
        # TODO: Initialize
        super().__init__(...)
        self.steering_gain = ...
        self.velocity_gain = ...
        self.target_velocity = ...

    def compute_control(
        self,
        car_state: CarState,
        perception_data: dict = None,
        costmap = None,
        plan: np.ndarray = None,
        dt: float = 0.1,
    ) -> dict:
        """
        Compute control commands.

        TODO:
        1. Check if enabled, return zero control if not
        2. If no plan or empty plan, return zero control
        3. Get car position: car_state.position()
        4. Find closest waypoint in plan:
           - Calculate distances: np.linalg.norm(plan - car_pos, axis=1)
           - Find index of minimum distance: np.argmin(distances)
        5. Get target waypoint (next waypoint after closest, or last if at end)
        6. Calculate steering:
           - Vector to target: target - car_pos
           - Angle to target: np.arctan2(vector[1], vector[0])
           - Angle error: target_angle - car_state.heading (normalize to [-pi, pi])
           - Desired steering: steering_gain * angle_error
           - Steering rate: (desired_steering - car_state.steering_angle) / dt
        7. Calculate velocity control:
           - Velocity error: target_velocity - car_state.velocity
           - Acceleration: velocity_gain * velocity_error
        8. Return {'acceleration': acc, 'steering_rate': steering_rate}
        """
        # TODO: Check if enabled
        if not ... or plan is None or len(plan) == 0:
            return {"acceleration": 0.0, "steering_rate": 0.0}

        # TODO: Get car position

        # TODO: Find closest waypoint
        closest_idx = ...

        # TODO: Get target waypoint (next one, or last if at end)
        target_idx = min(closest_idx + 1, len(plan) - 1)
        plan[target_idx]

        # TODO: Calculate steering
        angle_error = ...
        # Normalize angle error to [-pi, pi]
        angle_error = np.arctan2(np.sin(angle_error), np.cos(angle_error))

        steering_rate = ...

        # TODO: Calculate velocity control
        acceleration = ...

        return {"acceleration": acceleration, "steering_rate": steering_rate}

print("✓ Simple proportional controller structure created!")
print("  Now fill in the TODOs!")

## Step 3: Test Your Controller

In [ ]:
# Setup test
track = Track.create_simple_track(length=80.0, width=40.0, track_width=5.0)
planner = TrackPlanner(track, lookahead_distance=50.0)

start_point, start_heading = track.get_point_at_distance(0.0)
car_state = CarState(x=start_point[0], y=start_point[1], heading=start_heading, velocity=8.0)

# Generate plan
plan = planner.plan(car_state)

# TODO: Create your controller
my_controller = SimpleProportionalController(
    steering_gain=2.0,
    velocity_gain=0.5,
    target_velocity=10.0,
)

# TODO: Compute control
control = my_controller.compute_control(car_state, plan=plan, dt=0.1)

print("✓ Control computed:")
print(f"  Acceleration: {control['acceleration']:.3f} m/s²")
print(f"  Steering rate: {control['steering_rate']:.3f} rad/s")

In [ ]:
# Visualize controller behavior
from simple_autonomous_car.visualization import plot_car

fig, ax = plt.subplots(figsize=(12, 10))

track.visualize(ax=ax, frame="global")
planner.visualize(ax=ax, car_state=car_state, plan=plan, frame="global", color="green", label="Plan", show_waypoints=True)

car = Car(initial_state=car_state)
plot_car(car, ax=ax, show_heading=True)

# Show target waypoint
distances = np.linalg.norm(plan - car_state.position(), axis=1)
closest_idx = np.argmin(distances)
target_idx = min(closest_idx + 1, len(plan) - 1)
target = plan[target_idx]
ax.plot(target[0], target[1], "ro", markersize=10, label="Target Waypoint")
ax.plot([car_state.x, target[0]], [car_state.y, target[1]], "r--", alpha=0.5, label="Target Vector")

ax.set_title("Simple Proportional Controller")
ax.legend()
plt.show()

print("✓ Visualization complete!")

## Step 4: Build a Lookahead Controller

Now let's build a controller that looks ahead a certain distance on the path.

In [ ]:
class LookaheadController(BaseController):
    """
    A controller that uses a lookahead point on the path.

    TODO: Fill in the implementation!
    """

    def __init__(
        self,
        lookahead_distance: float = 10.0,
        steering_gain: float = 1.5,
        target_velocity: float = 10.0,
        velocity_gain: float = 0.5,
        name: str = "lookahead_controller",
    ):
        """
        Initialize the lookahead controller.

        TODO: Store all parameters
        """
        # TODO: Initialize
        super().__init__(...)
        self.lookahead_distance = ...
        self.steering_gain = ...
        self.target_velocity = ...
        self.velocity_gain = ...

    def compute_control(
        self,
        car_state: CarState,
        perception_data: dict = None,
        costmap = None,
        plan: np.ndarray = None,
        dt: float = 0.1,
    ) -> dict:
        """
        Compute control using lookahead point.

        TODO:
        1. Check if enabled and plan exists
        2. Find closest point on path
        3. Calculate cumulative distances along path from closest point
        4. Find point at lookahead_distance ahead:
           - Loop through waypoints from closest_idx
           - Accumulate distance along path
           - When distance >= lookahead_distance, use that waypoint
        5. Calculate steering towards lookahead point (similar to SimpleProportionalController)
        6. Calculate velocity control
        7. Return control commands
        """
        # TODO: Check if enabled
        if not ... or plan is None or len(plan) == 0:
            return {"acceleration": 0.0, "steering_rate": 0.0}

        # TODO: Get car position

        # TODO: Find closest point
        closest_idx = ...

        # TODO: Find lookahead point
        path_distance = 0.0
        lookahead_idx = closest_idx

        for i in range(closest_idx, len(plan) - 1):
            path_distance += ...
            if path_distance >= ...:
                lookahead_idx = i + 1
                break
        else:
            # If we didn't find a point far enough, use last waypoint
            lookahead_idx = len(plan) - 1

        plan[lookahead_idx]

        # TODO: Calculate steering (similar to SimpleProportionalController)
        angle_error = ...
        angle_error = np.arctan2(np.sin(angle_error), np.cos(angle_error))

        desired_steering = ...
        steering_rate = (desired_steering - car_state.steering_angle) / dt

        # TODO: Calculate velocity control
        acceleration = ...

        return {"acceleration": acceleration, "steering_rate": steering_rate}

print("✓ Lookahead controller structure created!")
print("  Now fill in the TODOs!")

In [ ]:
# Test lookahead controller
my_lookahead_controller = LookaheadController(
    lookahead_distance=15.0,
    steering_gain=1.5,
    target_velocity=10.0,
)

control = my_lookahead_controller.compute_control(car_state, plan=plan, dt=0.1)

print("✓ Lookahead control computed:")
print(f"  Acceleration: {control['acceleration']:.3f} m/s²")
print(f"  Steering rate: {control['steering_rate']:.3f} rad/s")

## Summary

Congratulations! You've built two different controllers! 🎉

### What You Learned

1. ✅ **Controller interface**: Understanding `BaseController` and `compute_control()`
2. ✅ **Proportional control**: Steering based on angle error
3. ✅ **Lookahead control**: Using a point ahead on the path
4. ✅ **Velocity control**: Maintaining target velocity

### Key Concepts

- **Steering rate**: Rate of change of steering angle (rad/s)
- **Acceleration**: Change in velocity (m/s²)
- **Lookahead distance**: How far ahead to look on the path
- **Angle error**: Difference between desired and current heading

### Next Steps

- Add adaptive lookahead (based on velocity)
- Implement PID control for smoother steering
- Add costmap-aware velocity control
- Implement Pure Pursuit (see [Building Controller](../building/building_controller.ipynb))